In [1]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: AMD Ryzen 9 3900X 12-Core Processor
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, znver2)
Environment:
  JULIA_PKG_SERVER = pkg.julialang.org


In [ ]:
using Pkg; Pkg.activate(".."); Pkg.instantiate();

In [3]:
using DatasetManager, C3D, LabDataSources, Biomechanics, Peaks, PlotlyJS, ProgressMeter, DSP, Statistics,
    HypothesisTests, GaitSymmetry, DataFrames, Query, PrettyTables, Distributions;

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-17548410990528892050\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16466571860003375651\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-17548410990528892050\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [4]:
datadir = joinpath(@__DIR__, "..", "data")
subsets = [
    DataSubset("c3d", Source{C3DFile}, datadir, "*.c3d"),
]

conds = TrialConditions((:sym,), Dict(:sym => r"a?sym"))

# Read all perturbations
trials = findtrials(subsets, conds; subject_fmt=r"(?<=S)(?<subject>\d+)")
summarize(trials)

Subjects:
 └ 15: 1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
Trials:
 ├ Number of trials: 30
 └ Number of trials per subject:
   └ 2: 15/15 (100%)
Conditions:
 ├ Observed levels:
 │ └ sym => ["asym", "sym"]
 └ Unique level combinations observed: 2 (full factorial)
      sym │ # trials
    ──────┼──────────
     asym │ 15
      sym │ 15
Sources:
 └ "c3d" => Source{C3DFile}, 30 trials (100%)


In [5]:
function heelheight(file::C3DFile; lheelmkr="LHEE", rheelmkr="RHEE", VT=3)
    fs = file.groups[:POINT][Int, :RATE]
    n = 4
    bw2 = Butterworth(n)
    corrfac = inv((2^inv(n)-1)^(1/4)) # Correction factor for Fc of multi-pass filters
    lpf = Lowpass(15*corrfac; fs)
    digfilt = digitalfilter(lpf, bw2)

    lheel = file.point[lheelmkr]
    rheel = file.point[rheelmkr]

    frheel = filtfilt(digfilt, rheel[:,VT])
    flheel = filtfilt(digfilt, lheel[:,VT])

    return flheel, frheel
end

heelheight (generic function with 1 method)

In [6]:
function predictgaitevents(trial)
    c3dsrc = readsource(trial, "c3d"; strip_prefixes=true)
    fs = c3dsrc.groups[:POINT][Int, :RATE]
    
    lheel, rheel = heelheight(c3dsrc)
    lfcpred, _ = peakproms!(argminima(lheel, 10), lheel; minprom=100)
    rfcpred, _ = peakproms!(argminima(rheel, 10), rheel; minprom=100)
    
    lheel_vel = centraldiff(lheel; dt=inv(fs), padding=ForwardBackwardPad())
    rheel_vel = centraldiff(rheel; dt=inv(fs), padding=ForwardBackwardPad())
    
    lfopred, _ = peakproms!(argmaxima(lheel_vel, 10), lheel_vel; minprom=1500)
    rfopred, _ = peakproms!(argmaxima(rheel_vel, 10), rheel_vel; minprom=1500)
    
    return Dict("LFC" => totimes(lfcpred, fs), "RFC" => totimes(rfcpred, fs),
        "LFO" => totimes(lfopred, fs), "RFO" => totimes(rfopred, fs))
end

predictgaitevents (generic function with 1 method)

In [7]:
function analyze(trial)
    events = predictgaitevents(trial)

    lfc = filter(>(25), events["LFC"])
    lfo = filter(>(25), events["LFO"])
    rfo = filter(>(25), events["RFO"])
    rfc = filter(>(25), events["RFC"])

    lswing = swing(lfc, lfo)[1:140]
    rswing = swing(rfc, rfo)[1:140]

    seg = Segment(trial, "c3d")
    sr = SegmentResult(seg)
    res = results(sr)

    res["lswing"] = lswing
    res["rswing"] = rswing
    res["lswing_avg"] = mean(lswing)*100
    res["rswing_avg"] = mean(rswing)*100
    res["lswing_cov"] = std(lswing)/mean(lswing)*100
    res["rswing_cov"] = std(rswing)/mean(rswing)*100

    return sr
end

analyze (generic function with 1 method)

In [8]:
srs = analyzedataset(analyze, trials, Source{C3DFile});

Analyzing trials... 100%|███████████████████████████████| Time: 0:00:11


In [9]:
df = unstack(DatasetManager.stack(srs, conds; variables=("lswing_avg","rswing_avg")))
df = disallowmissing(df)
metrics = [Sel86, Rob87, Vag92, Plo05, abs(Plo05), Zif08, Roc14, abs(Roc14), Que20, Alv20, Alv20b]
for metric in metrics
    if metric === Alv20b
        df[!,string(metric)] = Alv20.(df[!,"lswing_avg"],df[!,"rswing_avg"], Ref(sqrt((var(df[!,"lswing_avg"]; corrected=false) + var(df[!,"rswing_avg"]; corrected=false))/2)))
    else
        df[!,string(metric)] = metric.(df[!,"lswing_avg"],df[!,"rswing_avg"])
    end
end
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,subject,,1,,15,0,"CategoricalValue{Int64, UInt32}"
2,sym,,asym,,sym,0,"CategoricalValue{String, UInt32}"
3,lswing_avg,39.5419,37.7915,39.2097,43.8045,0,Float64
4,rswing_avg,37.5397,34.9171,38.085,40.4902,0,Float64
5,Sel86,0.951135,0.802183,0.957199,1.04208,0,Float64
6,Rob87,-5.22371,-21.953,-4.40005,4.12166,0,Float64
7,Vag92,-4.89696,-19.7817,-4.28012,4.03843,0,Float64
8,Plo05,-5.23212,-22.0418,-4.40136,4.12224,0,Float64
9,_abs_Plo05,6.20251,0.114162,5.42232,22.0418,0,Float64


In [10]:
optinv(x) = x < 1 ? inv(x) : x
summarystats = @from t in df begin
    @group (;t.lswing_avg, t.rswing_avg) by (;t.sym) into gt
    @orderby descending(key(gt).sym)
    @select {key(gt).sym, lswing_avg=mean(gt.lswing_avg), lswing_std=std(gt.lswing_avg), rswing_avg=mean(gt.rswing_avg), rswing_std=std(gt.rswing_avg),
        ratio=optinv(mean(gt.rswing_avg./mean(gt.lswing_avg))), ratio_std=std(gt.lswing_avg./mean(gt.lswing_avg))}
    @collect DataFrame
end
formatter = (v,i,j) -> !(v isa AbstractFloat) ? v : round(v, sigdigits=3)
pretty_table(summarystats; backend=Val(:html), standalone=false, formatters=formatter)

sym,lswing_avg,lswing_std,rswing_avg,rswing_std,ratio,ratio_std
"CategoricalArrays.CategoricalValue{String, UInt32}",Float64,Float64,Float64,Float64,Float64,Float64
sym,38.6,0.666,38.9,0.674,1.01,0.0172
asym,40.5,1.34,36.2,1.09,1.12,0.0331


In [11]:
grpd_avgs = @from t in df begin
    @group (;t.lswing_avg, t.rswing_avg) by (;t.sym) into gt
    @select {key(gt).sym, gt.lswing_avg, gt.rswing_avg}
    @collect DataFrame
end;

In [12]:
BartlettTest.(reshape.(grpd_avgs[!,"lswing_avg"], 15, 1), reshape.(grpd_avgs[!,"rswing_avg"], 15, 1))

2-element Vector{BartlettTest}:
 Bartlett's Test for Equality of Covariance Matrices
---------------------------------------------------
Population details:
    parameter of interest:   Equality of covariance matrices
    value under h_0:         NaN
    point estimate:          NaN

Test summary:
    outcome with 95% confidence: fail to reject h_0
    one-sided p-value:           0.4419

Details:
    number of observations: (15, 15)
    number of variables:    1
    χ² statistic:           0.591397
    degrees of freedom:     1

 Bartlett's Test for Equality of Covariance Matrices
---------------------------------------------------
Population details:
    parameter of interest:   Equality of covariance matrices
    value under h_0:         NaN
    point estimate:          NaN

Test summary:
    outcome with 95% confidence: fail to reject h_0
    one-sided p-value:           0.9634

Details:
    number of observations: (15, 15)
    number of variables:    1
    χ² statistic:           

In [13]:
OneSampleADTest.(grpd_avgs[!,"lswing_avg"], fit.(Normal, grpd_avgs[!,"lswing_avg"]))

2-element Vector{OneSampleADTest}:
 One sample Anderson-Darling test
--------------------------------
Population details:
    parameter of interest:   not implemented yet
    value under h_0:         NaN
    point estimate:          NaN

Test summary:
    outcome with 95% confidence: fail to reject h_0
    one-sided p-value:           0.4626

Details:
    number of observations:   15
    sample mean:              40.46488770808889
    sample SD:                1.3401653900224888
    A² statistic:             0.8229699813623393

 One sample Anderson-Darling test
--------------------------------
Population details:
    parameter of interest:   not implemented yet
    value under h_0:         NaN
    point estimate:          NaN

Test summary:
    outcome with 95% confidence: fail to reject h_0
    one-sided p-value:           0.8522

Details:
    number of observations:   15
    sample mean:              38.6188570845425
    sample SD:                0.6658082105389127
    A² statistic: 

In [14]:
OneSampleADTest.(grpd_avgs[!,"rswing_avg"], fit.(Normal, grpd_avgs[!,"rswing_avg"]))

2-element Vector{OneSampleADTest}:
 One sample Anderson-Darling test
--------------------------------
Population details:
    parameter of interest:   not implemented yet
    value under h_0:         NaN
    point estimate:          NaN

Test summary:
    outcome with 95% confidence: fail to reject h_0
    one-sided p-value:           0.5615

Details:
    number of observations:   15
    sample mean:              36.15907483335835
    sample SD:                1.086244706171326
    A² statistic:             0.6936855998548672

 One sample Anderson-Darling test
--------------------------------
Population details:
    parameter of interest:   not implemented yet
    value under h_0:         NaN
    point estimate:          NaN

Test summary:
    outcome with 95% confidence: fail to reject h_0
    one-sided p-value:           0.5493

Details:
    number of observations:   15
    sample mean:              38.920303767400256
    sample SD:                0.6741845998787865
    A² statistic:

In [15]:
grpd_metrics = @from t in df begin
    @orderby t.subject
    @group (;t.Sel86, t.Rob87, t.Vag92, t.Plo05, t._abs_Plo05, t.Zif08, t.Roc14, t._abs_Roc14, t.Que20, t.Alv20, t.Alv20b) by (;t.sym) into gt
    @orderby key(gt).sym
    @select {key(gt).sym, sel86=collect(gt.Sel86), rob87=collect(gt.Rob87), vag92=collect(gt.Vag92), plo05=collect(gt.Plo05), absplo05=collect(gt._abs_Plo05),
        zif08=collect(gt.Zif08), roc14=collect(gt.Roc14), absroc14=collect(gt._abs_Roc14), que20=collect(gt.Que20), alv20=collect(gt.Alv20), alv20b=collect(gt.Alv20b)}
    @collect DataFrame
end;

In [16]:
function CohensD(test::EqualVarianceTTest; unbiased=true)
    d = test.t*sqrt(inv(test.n_x) + inv(test.n_y))
    if unbiased
        d *= 1-(3/(4*test.df-1))
    end
    
    return d
end

CohensD (generic function with 1 method)

In [17]:
pairedt = Vector{OneSampleTTest}(undef, length(metrics))
betweent = Vector{EqualVarianceTTest}(undef, length(metrics))
metrics = names(grpd_metrics, Not("sym"))

for (i, metric) in enumerate(metrics)
    pairedt[i] = OneSampleTTest(only(grpd_metrics[in.(grpd_metrics.sym, Ref(["asym"])), metric]), only(grpd_metrics[in.(grpd_metrics.sym, Ref(["sym"])), metric]))
    betweent[i] = EqualVarianceTTest(only(grpd_metrics[in.(grpd_metrics.sym, Ref(["asym"])), metric]), only(grpd_metrics[in.(grpd_metrics.sym, Ref(["sym"])), metric]))
end

In [18]:
metrics_stats = DataFrame(metrics = metrics,
    xbar = getfield.(pairedt, :xbar), ci_low = get.(confint.(pairedt),1,NaN), ci_high = get.(confint.(pairedt),2,NaN), t = getfield.(pairedt, :t), d = abs.(CohensD.(betweent)))
pretty_table(metrics_stats, backend=Val(:html), standalone=false, formatters = ft_printf("%.3g"))

metrics,xbar,ci_low,ci_high,t,d
String,Float64,Float64,Float64,Float64,Float64
sel86,-0.114,-0.131,-0.0963,-14.2,4.03
rob87,-12,-14,-10,-13,3.8
vag92,-11.3,-13.1,-9.61,-14.1,4.03
plo05,-12,-14,-10,-12.9,3.79
absplo05,10.1,7.47,12.7,8.29,3.25
zif08,-3.82,-4.44,-3.19,-13,3.81
roc14,-4.61,-5.39,-3.82,-12.6,3.71
absroc14,3.86,2.83,4.88,8.09,3.17
que20,-0.113,-0.131,-0.0961,-14.1,4.03
